In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

In [ ]:
import yaml
from ultralytics import YOLO

import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pathlib
%matplotlib inline
from collections import Counter

In [ ]:
!cat ../input/apple-quality-dataset/Apple_Detection.v1i.yolov8/data.yaml

In [ ]:
# Load the YAML file
with open('../input/apple-quality-dataset/Apple_Detection.v1i.yolov8/data.yaml', 'r') as f:
    data_yaml = yaml.safe_load(f)

# update YAML with absolute path to kaggle data. You must use absolute path, relative path won't work
data_yaml['train'] = '/kaggle/input/apple-quality-dataset/Apple_Detection.v1i.yolov8/train/images'
data_yaml['val'] = '/kaggle/input/apple-quality-dataset/Apple_Detection.v1i.yolov8/valid/images'

# write to disk
with open('data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

In [ ]:
data_dir = Path("../input/apple-quality-dataset/Apple_Detection.v1i.yolov8")
images_dir = data_dir / "train" / "images"
labels_dir = data_dir / "train" / "labels"

class_names = data_yaml['names']

# Read the image file paths and annotations
image_paths = list(images_dir.glob("*.jpg"))
label_paths = sorted(labels_dir.glob("*.txt"))

In [ ]:
resolutions = []

for image_path in image_paths:
    img = cv2.imread(str(image_path))
    h, w, _ = img.shape
    resolutions.append((w, h))

unique_resolutions = set(resolutions)
print("Unique resolutions:", unique_resolutions)

In [ ]:
# Number of images to randomly select
num_images = 5

# Get the list of all image files in the 'images' directory
image_files = [f for f in pathlib.Path(images_dir).iterdir() if f.is_file()]

# Shuffle the list of image files
random.shuffle(image_files)

# Select the specified number of image files
selected_image_files = image_files[:num_images]

for selected_image_file in selected_image_files:
    demo_image = selected_image_file
    # Get the corresponding label file
    demo_label = pathlib.Path(labels_dir) / f"{selected_image_file.stem}.txt"

    # Load the image using OpenCV's imread function
    image = cv2.imread(str(demo_image))

    # Get the list of class names from the 'data' dictionary
    class_list = data_yaml['names']

    # Define a list of colors to be used to draw bounding boxes
    colors = [(0,189,255)] 

    # Get the height and width of the image
    height, width, _ = image.shape

    # Create an empty list T
    T = []

    # Open the label file 'demo_label' in read mode and process each line
    with open(demo_label, "r") as file1:
        for line in file1.readlines():
            # Split the line into a list of strings
            split = line.split(" ")

            # Get the class id from the first element of the split list
            class_id = int(split[0])

            # Get the color corresponding to the class id from the 'colors' list
            color = colors[class_id]
            clazz = class_list[class_id]

            # Get the x, y, w, h bounding box coordinates from the split list
            x, y, w, h = float(split[1]), float(split[2]), float(split[3]), float(split[4])

            # Rescale the x, y, w, h values to the size of the image
            box = [int((x - 0.5*w)* width), int((y - 0.5*h) * height), int(w*width), int(h*height)]

            # Draw a rectangle on the image using the 'box' and 'color' values
            cv2.rectangle(image, box, color, 2)

            # Draw a filled rectangle for the class label on the image
            cv2.rectangle(image, (box[0], box[1] - 20), (box[0] + box[2], box[1]), color, -1)

            # Write the class label on the image
            cv2.putText(image, class_list[class_id], (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,0))

        # Show the image using matplotlib
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        # Optionally resize the image (commented out in code)
        image = cv2.resize(image, (600, 600))
        plt.show()

In [ ]:
baseline_model = YOLO('yolov8x.pt')

In [ ]:
params = {
    'data':'data.yaml',
    'imgsz':640,
    'epochs':50,
    'optimizer':'auto',
    'pretrained':True,
    'lr0':0.01,
    'lrf': 0.01,
    'cos_lr': False
}

In [ ]:
results = baseline_model.train(**params)

In [ ]:
results.results_dict

In [ ]:
!ls runs/detect/train2/val_batch*.jpg

In [ ]:
params['hsv_h'] = 0.02
params['hsv_s'] = 0.50
params['hsv_v'] = 0.30
params['degrees'] = 5.0
params['translate'] = 0.05
params['scale'] = 0.3
params['shear'] = 0.0
params['perspective'] = 0.0
params['flipud'] = 0.0
params['fliplr'] = 0.30
params['mosaic'] = 0.50
params['mixup'] = 0.0
params['label_smoothing'] = 0.15
#params['fl_gamma'] = 2
params['box'] = 10.0
params['cls'] = 2.0
params['dfl'] = 2.0

In [ ]:
del params['fl_gamma']

In [ ]:
params

In [ ]:
yolo_tuned =  YOLO('runs/detect/train/weights/best.pt')
results_1 = yolo_tuned.train(**params)

In [ ]:
results_1.results_dict

In [ ]:
# Perform inference on an image
results = model('/kaggle/input/apple-quality-dataset/Apple_Detection.v1i.yolov8/valid/images/image.jpg')

# Visualize the results
for result in results:
    boxes = result.boxes  # Bounding boxes
    
    # Get the image
    img = result.orig_img
    
    # Plot the bounding boxes
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # get box coordinates
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
!ls runs/detect/train/